In [1]:
# Install dependencies
!pip install fastapi uvicorn pyngrok torch torchvision Pillow opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00:00


In [2]:
!pip install python-multipart

In [3]:
import base64
import io
import cv2
import numpy as np
import torch
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from PIL import Image
from torch import nn
from torchvision import transforms
from torchvision.models import mobilenet_v2
from pyngrok import ngrok
import nest_asyncio
import uvicorn

In [4]:
# Helper function: Convert CV2 image to Base64
def cv2_image_to_base64(image):
    _, buffer = cv2.imencode(".jpg", image)
    return base64.b64encode(buffer).decode("utf-8")


# Define the model class
class ObjectDetectionModel(nn.Module):
    def __init__(self):
        super(ObjectDetectionModel, self).__init__()
        self.backbone = mobilenet_v2(weights="DEFAULT").features
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Linear(512, 5),
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        is_object = torch.sigmoid(x[:, :1])
        bbox = torch.sigmoid(x[:, 1:])
        return is_object, bbox


# Initialize the FastAPI app
app = FastAPI()

# Load the model
model = ObjectDetectionModel()
# Replace 'best_object_detection.pt' with the correct path to your model file
# Example: model.load_state_dict(torch.load('/content/best_object_detection.pt', map_location=torch.device('cpu')))
model.load_state_dict(torch.load("best_object_detection.pt", map_location=torch.device("cpu")))
model.eval()

# Image transformation
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 42.2MB/s]
<ipython-input-4-9e199ec8a311>:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of t

In [6]:
@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    try:
        image = Image.open(io.BytesIO(await file.read())).convert("RGB")
        cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        img_width, img_height = image.size
        image_tensor = transform(image).unsqueeze(0)

        with torch.no_grad():
            is_object, bbox = model(image_tensor)

        is_object_value = is_object.item()
        bbox = bbox.squeeze(0).tolist()
        x, y, w, h = bbox
        x = x * img_width
        y *= img_height
        w *= img_width
        h *= img_height
        x1, y1, x2, y2 = int(x - w / 2), int(y - h / 2), int(x + w / 2), int(y + h / 2)
        cv2.rectangle(cv_image, (x1, y1), (x2, y2), (0, 255, 0), 1)

        if is_object_value > 0.5:
            response = {
                "is_object": True,
                "bbox": {"x": x, "y": y, "w": w, "h": h},
                "image_base64": cv2_image_to_base64(cv_image),
            }
        else:
            response = {"is_object": False, "bbox": None}

        return JSONResponse(content=response)

    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

@app.get("/")
def root():
    return {"message": "Single Object Detection API is running!"}

In [7]:
# Tambahkan token ngrok Anda
!ngrok config add-authtoken YOUR_NGROK_AUTHTOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
# Run the FastAPI app using ngrok
if __name__ == "__main__":
    # Allow nested event loops in Colab
    nest_asyncio.apply()

    # Create ngrok tunnel
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url}")

    # Start the server
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [240]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://bbba-35-221-188-31.ngrok-free.app" -> "http://localhost:8000"
INFO:     103.184.52.38:0 - "GET / HTTP/1.1" 200 OK
INFO:     103.184.52.38:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     103.184.52.38:0 - "GET / HTTP/1.1" 200 OK
INFO:     103.184.52.38:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     103.184.52.38:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     103.184.52.38:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     103.184.52.38:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     103.184.52.38:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     103.184.52.38:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     103.184.52.38:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [240]
